# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import scipy as sp
from scipy import optimize



# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
import HouseholdSpecializationModel

# Question 1

**Explain how you solve the model**

In [ ]:
# code for solving the model (remember documentation and comments)
model = HouseholdSpecializationModel.HouseholdSpecializationModelClass()

HM = np.zeros([3,3]) # empty matrix to be filled
alphalist = [0.25, 0.50, 0.75] #starting alpha values
sigmalist = [0.5, 1.0, 1.5] # starting sigma values

# iterate over alphalist and then sigma list and call solution
for i, j in enumerate(alphalist): 
    model.par.alpha = j
    for it, s in enumerate(sigmalist):
        model.par.sigma = s
        out =  model.solve_discrete()
        HM[i,it] = out.HF /out.HM



In [ ]:
print(HM) # prints the solution to question 1 - update to be nicer

In [ ]:
#force reset of parameters to standard
model.alpha = 0.5
model.sigma=1

# Question 2

In [ ]:
model = HouseholdSpecializationModel.HouseholdSpecializationModelClass()
model.solve_wF_vec(discrete=True)

In [ ]:
# plot the log ratio of hours worked for female and male and the log ratio of wage for female and male
y = np.log(model.sol.HF_vec/model.sol.HM_vec) # call the solution and make fraction of hours and take log
print(y)
x = np.log(model.par.wF_vec) #call the parameters from the class 
print(x)
plt.plot(x,y)

Explain your code and procedure

# Question 3

Explain your code and procedure

In [ ]:
# code

# call model
model.solve_wF_vec(discrete=False)

# calculate new ratios 
y1 = np.log(model.sol.HF_vec/model.sol.HM_vec)
x1 = np.log(model.par.wF_vec)

# plot
plt.plot(x1, y1)

print(y1)
print(x1)

# Question 4


ADD CONCISE CONLUSION.